<a href="https://colab.research.google.com/github/silviolima07/Kaggle/blob/master/Analise_Viagens_Onibus_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://www.kaggle.com/code/tientd95/advanced-pyspark-for-exploratory-data-analysis
# https://pub.towardsai.net/exploratory-data-analysis-eda-using-pyspark-b43e71fcec9f
# https://medium.com/@aieeshashafique/exploratory-data-analysis-using-pyspark-dataframe-in-python-bd55c02a2852

![](https://drive.google.com/uc?export=view&id=119CJM4EnZ5XDikTz6_evjINNJBz9pt_2)

# Foram usados dados do Kaggle.
# Colunas foram selecionadas priorizando aquelas que podem ajudar entender o fluxo de bilhetes emitidos no ano de 2019. No kaggle a base de dados tem informações de 2019 e 2020. Porém os dados de 2020 foram filtrados e removidos pois foi o periodo marcado pelo início da Covid 19 que exigiu restrições de deslocamento e viagens foram impactadas.
# Gráficos:
- de barras que fácilmente podem ser compreendidos
- sankey, indicando o fluxo da origem ao destino
- tabelas, mostrando valores estatísticos calculados

# Dados usados:
- passagens/viagens - https://www.kaggle.com/datasets/danlessa/brazil-interstate-bus-travels?resource=download

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=afb50cde6a082abb979bfdd605459bf74eef23640bfd923c1ad71c0271533261
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
# Import other modules not related to PySpark
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# This helps auto print out the items without explixitly using 'print'
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [3]:
# Import PySpark related modules
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

MAX_MEMORY = '10G'
# Initialize a spark session.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Pyspark guide") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
df_03_2019 =  spark.read.options(header='True', inferSchema='True',delimiter=';').csv("/content/drive/MyDrive/Viagens/venda_passagem_03_2019/venda_passagem_03_2019.csv")
#
df_11_2019 =  spark.read.options(header='True', inferSchema='True',delimiter=';').csv("/content/drive/MyDrive/Viagens/venda_passagem_11_2019/venda_passagem_11_2019.csv")
#
df_12_2019 =  spark.read.options(header='True', inferSchema='True',delimiter=';').csv("/content/drive/MyDrive/Viagens/venda_passagem_12_2019/venda_passagem_12_2019.csv")

In [28]:
result = df_03_2019.union(df_11_2019)
df_full = result.union(df_12_2019)

In [29]:
df_full.show()

+-------------+--------------+-------------------------+--------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+-----------+-------------------+-----------+---------------+-------------------+--------------------+------------+-------------------------+-------------------+-------------+-------------------+-----------+--------------+---------------------+
|codigo_viagem|          cnpj|numero_equipamento_fiscal|numero_bilhete|data_emissao_bilhete|hora_emissao_bilhete|categoria_transporte|nu_linha|origem_destino_linha| ponto_origem_viagem|ponto_destino_viagem|        tipo_servico|data_viagem|        hora_viagem|tipo_viagem|numero_poltrona|plataforma_embarque|      tipo_gratitude|valor_tarifa|valor_percentual_desconto|valor_aliquota_icms|valor_pedagio|valor_taxa_embarque|valor_total|origem_emissao|in_passagem_cancelada|
+-------------+--------------+-------------------------+--

In [21]:
usecols = ['data_emissao_bilhete', 'categoria_transporte', 'origem_destino_linha', 'ponto_origem_viagem',
       'ponto_destino_viagem','tipo_servico', 'data_viagem', 'tipo_gratitude',
       'tipo_viagem', 'numero_poltrona', 'valor_total', 'in_passagem_cancelada']

In [31]:
df_viagem = df_full.select(*usecols)

In [32]:
df_viagem.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------+---------------+-----------+---------------------+
|data_emissao_bilhete|categoria_transporte|origem_destino_linha| ponto_origem_viagem|ponto_destino_viagem|        tipo_servico|data_viagem|      tipo_gratitude|tipo_viagem|numero_poltrona|valor_total|in_passagem_cancelada|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------+---------------+-----------+---------------------+
|          21-03-2019|       Interestadual|FLORIANOPOLIS(SC)...|JOINVILLE/SC        |SAO PAULO/SP        |Convencional com ...| 30-03-2019|Tarifa Normal - s...|    Regular|              2|      229.9|                  N�O|
|          15-03-2019|       Interestadual|FLORIANOPOLIS(SC)...|BALNEARIO PICARRA...|SAO PAULO/SP        |Co

In [ ]:
def value_counts_df(df, col):
    """
    Returns pd.value_counts() as a DataFrame

    Parameters
    ----------
    df : Pandas Dataframe
        Dataframe on which to run value_counts(), must have column `col`.
    col : str
        Name of column in `df` for which to generate counts

    Returns
    -------
    Pandas Dataframe
        Returned dataframe will have a single column named "count" which contains the count_values()
        for each unique value of df[col]. The index name of this dataframe is `col`.

    Example
    -------
    >>> value_counts_df(pd.DataFrame({'a':[1, 1, 2, 2, 2]}), 'a')
       count
    a
    2      3
    1      2
    """
    df = pd.DataFrame(df[col].value_counts())
    df.index.name = col
    df.columns = ['count']
    df = df.reset_index()
    df.columns = [col, 'counts'] # change column names
    return df

In [ ]:
def remove_outlier(df,col):
    # Calculate first and third quartile
    first_quartile = df[col].describe()['25%']
    third_quartile = df[col].describe()['75%']

    # Interquartile range
    iqr = third_quartile - first_quartile

    # Remove outliers
    data = df[(df[col] > (first_quartile - 3 * iqr)) &
            (df[col] < (third_quartile + 3 * iqr))]
    return data

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()

        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)

        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")

        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
def total_values(df):
  cols = list(df.columns)
  for col in cols:
    print("\n\nColuna:",col,"\n",df[col].value_counts())

In [ ]:
def rem_time(d):
    s = ''
    s = str(d.year) + '-' + str(d.month) + '-' + str(d.day)
    return s

In [ ]:
usecols = ['data_emissao_bilhete', 'categoria_transporte', 'origem_destino_linha', 'ponto_origem_viagem',
       'ponto_destino_viagem','tipo_servico', 'data_viagem', 'tipo_gratitude',
       'tipo_viagem', 'numero_poltrona', 'valor_total', 'in_passagem_cancelada']

In [ ]:
df_geral = pd.read_csv("/content/drive/MyDrive/Viagens/venda_passagem_12_2019/venda_passagem_12_2019.csv", sep=';',encoding = 'latin-1', on_bad_lines='skip', nrows=1000)

In [ ]:
df_geral.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
codigo_viagem,1000.0,NaN,NaN,NaN,2376394.88,36669.367435,2127415.0,2346375.0,2380521.0,2406671.0,2453938.0
cnpj,1000.0,NaN,NaN,NaN,48509726423133.585938,27924366567331.378906,1751730000197.0,23542573000142.0,60765633000112.0,76539600000194.0,95592077000104.0
numero_equipamento_fiscal,1000,167,35191261084018000103,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
numero_bilhete,1000.0,NaN,NaN,NaN,12266067.064,31569569.856572,0.0,140268.0,356717.5,2567598.0,100999454.0
data_emissao_bilhete,1000,72,20-12-2019,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hora_emissao_bilhete,1000,682,14:45:00,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categoria_transporte,1000,3,Interestadual,994,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nu_linha,1000.0,NaN,NaN,NaN,9567996.585,3693156.976446,3002100.0,8002800.0,8009700.0,11000678.5,20002600.0
origem_destino_linha,1000,217,SAO PAULO(SP) - RIO DE JANEIRO(RJ),153,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ponto_origem_viagem,1000,116,SAO PAULO/SP,230,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_geral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   codigo_viagem              1000 non-null   int64  
 1   cnpj                       1000 non-null   int64  
 2   numero_equipamento_fiscal  1000 non-null   object 
 3   numero_bilhete             1000 non-null   int64  
 4   data_emissao_bilhete       1000 non-null   object 
 5   hora_emissao_bilhete       1000 non-null   object 
 6   categoria_transporte       1000 non-null   object 
 7   nu_linha                   1000 non-null   int64  
 8   origem_destino_linha       1000 non-null   object 
 9   ponto_origem_viagem        1000 non-null   object 
 10  ponto_destino_viagem       1000 non-null   object 
 11  tipo_servico               1000 non-null   object 
 12  data_viagem                1000 non-null   object 
 13  hora_viagem                1000 non-null   object

In [ ]:
df_geral['valor_total'] = pd.to_numeric(df_geral['valor_total'], downcast='float')
#
df_geral['numero_poltrona'] = pd.to_numeric(df_geral['numero_poltrona'], downcast='unsigned')
#
df_geral['categoria_transporte'] = df_geral['categoria_transporte'].astype('category')
df_geral['origem_destino_linha'] = df_geral['origem_destino_linha'].astype('category')
df_geral['ponto_destino_viagem'] = df_geral['ponto_destino_viagem'].astype('category')
df_geral['tipo_gratitude'] = df_geral['tipo_gratitude'].astype('category')
df_geral['tipo_servico'] = df_geral['tipo_servico'].astype('category')
df_geral['in_passagem_cancelada'] = df_geral['in_passagem_cancelada'].astype('category')

In [ ]:
# Definir o tipo a ser usado na leitura dos datasets
dtypes = df_geral.dtypes
colnames = dtypes.index
types = [i.name for i in dtypes.values]
column_types = dict(zip(colnames, types))
#
parse_date = ['data_emissao_bilhete', 'data_viagem']

In [ ]:
nrow_12_2019 =  2500000 #2200000
nrow_11_2019 =  1000000
nrow_3_2019  =  1000000


# Colunas selecionadas e tipos de colunas definidos.
# Serão lidos 3 datasets com dados de bilhetes de viagens no ano de 2019.

In [ ]:
df_12_2019 = pd.read_csv("/content/drive/MyDrive/Viagens/venda_passagem_12_2019/venda_passagem_12_2019.csv", sep=';',encoding = 'latin-1', on_bad_lines='skip', dtype=column_types, parse_dates=parse_date, usecols=usecols, nrows=nrow_12_2019)

In [ ]:
df_12_2019.shape

(2500000, 12)

In [ ]:
df_11_2019 = pd.read_csv("/content/drive/MyDrive/Viagens/venda_passagem_11_2019/venda_passagem_11_2019.csv", sep=';',encoding = 'latin-1', on_bad_lines='skip', dtype=column_types, parse_dates=parse_date, usecols=usecols, nrows=nrow_11_2019)

In [ ]:
df_11_2019.shape

(1000000, 12)

In [ ]:
df_03_2019 = pd.read_csv("/content/drive/MyDrive/Viagens/venda_passagem_03_2019/venda_passagem_03_2019.csv", sep=';',encoding = 'latin-1', on_bad_lines='skip', dtype=column_types, parse_dates=parse_date, usecols=usecols, nrows=nrow_3_2019)

In [ ]:
df_03_2019.shape

In [ ]:
#df_viagem = pd.concat([df_03_2019, df_11_2019, df_12_2019, df_01_2020], axis=0)
df_viagem = pd.concat([df_03_2019, df_11_2019, df_12_2019], axis=0)

In [ ]:
df_viagem.shape

In [ ]:
df_viagem.info()

In [ ]:
df_viagem.head()

In [ ]:
df_viagem = df_viagem.loc[df_viagem.valor_total >= 0]

In [ ]:
temp = df_viagem['origem_destino_linha'].str.split(' - ',expand=True)


In [ ]:
temp[1]

In [ ]:
temp = df_viagem['origem_destino_linha'].str.split(' - ',expand=True)
df_viagem['origem_linha'] = temp[0]
df_viagem['destino_linha']= temp[1]

In [ ]:
df_viagem['origem_destino_linha'] = df_viagem.origem_destino_linha.str.strip().str.replace(' ','')

In [ ]:
orig = df_viagem['origem_linha'].str.split('(', expand=True)
orig.rename(columns={0: "cidade_origem_linha", 1:"uf_origem_linha" }, inplace=True)
orig['uf_origem_linha'] = orig.uf_origem_linha.str.replace(')','')


In [ ]:
dest = df_viagem['destino_linha'].str.split('(', expand=True)
dest.rename(columns={0: "cidade_destino_linha", 1:"uf_destino_linha" }, inplace=True)
dest['uf_destino_linha'] = dest.uf_destino_linha.str.replace(')','')

In [ ]:
df_viagem["cidade_origem_linha"] = orig.cidade_origem_linha.str.strip()
df_viagem['uf_origem_linha'] = orig.uf_origem_linha.str.strip()
#
df_viagem["cidade_destino_linha"] = dest.cidade_destino_linha.str.strip()
df_viagem['uf_destino_linha'] = dest.uf_destino_linha.str.strip()

In [ ]:
df_viagem['ponto_destino_viagem']= df_viagem.ponto_destino_viagem.str.rstrip().str.replace(' ', '-')

In [ ]:
df_viagem['tipo_gratitude'].loc[df_viagem.tipo_gratitude.isnull()]

In [ ]:
df_viagem.dropna(inplace=True)

In [ ]:
df_viagem.isnull().sum()

In [ ]:
df_viagem.columns

In [ ]:
df_temp = df_viagem.copy()

In [ ]:
df_temp.shape


# O número de assentos depende se tem banheiro ou tem dois andares.
# Sendo de dois andares tem no máximo 104 lugares.
- https://www.auditorioibirapuera.com.br/quantas-poltronas-tem-um-onibus-de-dois-andares/

---



In [ ]:
df_temp.numero_poltrona.value_counts()

# Linhas com numero de poltrona acima de 104 serão excluidas.

In [ ]:
df_temp[['ponto_origem_viagem', 'numero_poltrona']].loc[df_temp.numero_poltrona > 104]

In [ ]:
df_temp.shape

In [ ]:
df_temp = df_temp.loc[df_temp.numero_poltrona <= 104]

In [ ]:
df_temp.shape

In [ ]:
df_temp[['ponto_origem_viagem', 'numero_poltrona']]

In [ ]:
df_temp.numero_poltrona.value_counts().sort_values()

# Remover as linha onde a passagem foi cancelada.

In [ ]:
df_temp.in_passagem_cancelada.value_counts()

In [ ]:
print("Antes: ", df_temp.shape)
df_temp = df_temp.loc[df_temp.in_passagem_cancelada=='NÃO']
print("Depois:", df_temp.shape)

In [ ]:
df_temp =   df_temp.drop(columns=['in_passagem_cancelada'])

# Extrair os valores de dia, mês e ano da variavel data_emissao_bilhete.

In [ ]:
dia_emissao = df_temp.data_emissao_bilhete.dt.day
mes_emissao = df_temp.data_emissao_bilhete.dt.month
ano_emissao = df_temp.data_emissao_bilhete.dt.year
df_temp['dia_emissao'] = dia_emissao
df_temp['mes_emissao'] = mes_emissao
df_temp['ano_emissao'] = ano_emissao


# Extrair os valores de dia, mês e ano da variavel data_viagem.

In [ ]:
dia_viagem = df_temp.data_viagem.dt.day
mes_viagem = df_temp.data_viagem.dt.month
ano_viagem = df_temp.data_viagem.dt.year
df_temp['dia_viagem'] = dia_viagem
df_temp['mes_viagem_num'] = mes_viagem
df_temp['ano_viagem'] = ano_viagem

In [ ]:
dict_mes = {'1':"Jan", '2':'Fev', '3':'Mar', '4':'Abr', '5': 'Mai', '6': 'Jun', '7': 'Jul', '8': 'Ago', '9':'Set', '10':'Out', '11': 'Nov', '12': 'Dez'}

In [ ]:
dict_mes.get('1')

In [ ]:
df_temp.mes_viagem_num

In [ ]:
lista_mes = []
for i in (df_temp.mes_viagem_num):
  lista_mes.append(dict_mes.get(str(i)))

In [ ]:
#month_name = df4.data_viagem.dt.strftime("%B")
df_temp['mes_viagem'] = lista_mes

In [ ]:
df_temp[['mes_viagem', 'mes_viagem_num']]

# Extrair os valores que identificam o dia da semana.

In [ ]:
df_temp['semana_num'] = df_temp.data_viagem.dt.strftime("%w")

In [ ]:
dict_semana = {'0':"Dom", '1':"Seg", '2':"Ter",'3': "Qua",'4':"Qui", '5':"Sex", '6':"Sab"}

In [ ]:
lista_dias = []
for i in (df_temp.semana_num):
  lista_dias.append(dict_semana.get(str(i)))

In [ ]:
df_temp['dia_semana_viagem'] = lista_dias

In [ ]:
df_temp[['dia_emissao', 'mes_emissao', 'ano_emissao', 'dia_viagem', 'mes_viagem', 'ano_viagem', 'dia_semana_viagem']]

In [ ]:
df_temp = df_temp.loc[df_temp['ano_viagem'] == 2019]

In [ ]:
df_temp[['dia_emissao', 'mes_emissao', 'ano_emissao', 'dia_viagem', 'mes_viagem', 'ano_viagem', 'dia_semana_viagem']]

In [ ]:
dif_days= df_temp.data_viagem - df_temp.data_emissao_bilhete

In [ ]:
df_temp['dif_days'] = (dif_days.astype(str).str.replace("days",'')).astype('int')

In [ ]:
#df_temp[['dia_emissao', 'mes_emissao', 'ano_emissao', 'dia_viagem', 'mes_viagem', 'ano_viagem', 'dif_days']].loc[df_temp.dif_days <= 0]

In [ ]:
df_temp = df_temp.loc[df_temp.dif_days >= 0]

In [ ]:
df_temp.columns

# Filtrar por Destino

In [ ]:
temp = df_temp[['ponto_destino_viagem','data_emissao_bilhete','data_viagem'',
       'valor_total',
       'dia_semana_viagem']]
@interact
def filter_destino(Destino = list(temp.destino_linha.unique())):

    return temp[temp.destino_linha == Destino]

# Filtrar por Dia da Semana

In [ ]:
temp = df_temp[['dia_semana_viagem','ponto_origem_viagem','ponto_destino_viagem','data_emissao_bilhete','data_viagem'
       'valor_total',
       ]]
@interact
def filter_dia(Dia = list(temp.dia_semana_viagem.unique())):

    return temp[temp.dia_semana_viagem == Dia]

# Total Viagens por Dias da Semana

In [ ]:
fig = plt.figure(figsize=(5, 1.5), dpi = 200, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
temp_sem_viagem = value_counts_df(df_temp, 'dia_semana_viagem')
with sns.axes_style('white'):
    g = sns.barplot(x="dia_semana_viagem", y = 'counts', data=temp_sem_viagem)
    plt.title("Total de Viagens por Dia da Semana")
    plt.savefig("Fig1.png", bbox_inches='tight')

# Quais os top 10 destinos mais procurados?

In [ ]:
fig = plt.figure(figsize=(6, 2.5), dpi = 200, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
top_10 = df_temp['ponto_destino_viagem'].value_counts().iloc[:10].sort_values(ascending=True)
top_10.plot.barh(edgecolor='black', color='red');
plt.xlabel('Frequencia')
plt.ylabel('destinos')
plt.title("Top 10 Destinos");
plt.savefig("Fig2.png", bbox_inches='tight')

# Quais os top 10 origens?

In [ ]:
fig = plt.figure(figsize=(6, 2.5), dpi = 200, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
top_10 = df_temp['ponto_origem_viagem'].value_counts().iloc[:10].sort_values(ascending=True)
top_10.plot.barh(edgecolor='black', color='red');
plt.xlabel('Frequencia')
plt.ylabel('origem')
plt.title("Top 10 Origens");
plt.savefig("Fig3.png", bbox_inches='tight')

# Qual o total de viagens por mês no dataset gerado?

In [ ]:
fig = plt.figure(figsize=(6, 2.5), dpi = 200, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
ranking_meses= df_temp['mes_viagem'].value_counts()
#print("Total Viagens Mes\n",ranking_meses, '\n')
ranking_meses.plot(kind='bar', edgecolor='black', rot=4, color='red');
plt.xlabel('Meses')
plt.ylabel('Frequencia')
plt.title("Ranking Meses");
plt.savefig("Fig4.png", bbox_inches='tight')

# Total de Bilhetes Emitidos por Gratitude

In [ ]:
temp = df_temp['tipo_gratitude'].str.split('-', expand=True)
print('\033[1m','\nTipo\t\t\t\t\t\tTotal\n\n'+str(temp[0].value_counts()))

In [ ]:
fig = plt.figure(figsize=(7, 2.5), dpi = 100, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
temp = df_temp['tipo_gratitude'].str.split('-', expand=True)
gratitude_count = temp[0]
gratitude= gratitude_count.value_counts().sort_values(ascending=True)
#print("Total Tipos de Gratitude\n",gratitude, '\n')
gratitude.plot.barh(edgecolor='black', color='red');
plt.xlabel('Frequencia')
plt.ylabel('Tipo de Gratitude')
plt.title("Total Tipo de Gratitude");
plt.savefig("Fig5.png",  bbox_inches='tight') # do not permit cut text

# Total de Emissoes de Bilhetes por Dia

In [ ]:
df_temp.columns

In [ ]:
temp_data = []
for i in df_temp.data_emissao_bilhete:
  temp_data.append(rem_time(i))

df_temp['data_emissao_bilhete'] = temp_data


In [ ]:
fig = plt.figure(figsize=(6, 2.5), dpi = 150, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
temp2 = df_temp['data_emissao_bilhete'].value_counts().iloc[:10]
temp2.plot.barh(x= temp2.index , y=temp.values, edgecolor='black', color='red');
plt.xlabel('Frequencia')
plt.ylabel('Data Emissao Bilhete')
plt.title("Top 10\nMaiores Emissoes de Bilhetes por Dia\n");
plt.savefig("Fig6.png",  bbox_inches='tight')

# Qual o total de Passagens Vendidas entre UFs?

In [ ]:
df_uf = df_temp[['uf_origem_linha', 'uf_destino_linha']].value_counts().reset_index(name='count')
#
df_uf['ufs'] = df_uf.uf_origem_linha+'-->'+df_uf.uf_destino_linha

In [ ]:
fig = plt.figure(figsize=(6, 2.5), dpi = 150, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
temp = df_uf.iloc[:10]
#temp.plot.bar(x = 'ufs' , y= 'count', edgecolor='black', rot=80, color='red');
sns.barplot(x=temp['count'], y=temp['ufs'], orient='h', color='red')
plt.xlabel('Frequencia')
plt.ylabel('Origem-Destino')
plt.title("Top 10\nTotal de Bilhetes Emitidos Entre Ufs\nOrigem-->Destino\n");
plt.savefig("Fig7.png",  bbox_inches='tight')

# Fluxo de Viagens entre UFs
- https://python-graph-gallery.com/basic-sankey-diagram-with-pysankey/

In [ ]:
filter=10000
df_filtered = df_uf.loc[df_uf['count']>= filter].sort_values(by='count', ascending=False)
df_filtered.shape

In [ ]:
sankey(
    left=df_filtered["uf_origem_linha"], right=df_filtered["uf_destino_linha"],
    leftWeight= df_filtered["count"], rightWeight=df_filtered["count"],
    aspect=20, fontsize=10
)

# Get current figure
fig = plt.gcf()



# Set size in inches
fig.set_size_inches(10, 10)

# Set the color of the background to white
fig.set_facecolor("#EFE9E6")
plt.title("Total de Bilhetes Emitidos entre UFs\nOrigem --> Destino\nFilter: Acima de 10k");
# Save the figure
plt.subplots_adjust(top=0.88)
fig.savefig("fluxo_passageiros.png", bbox_inches="tight", dpi=180)
plt.savefig("Fig8.png",  bbox_inches='tight')



In [ ]:
df_uf2 = df_temp[['uf_origem_linha', 'uf_destino_linha', 'mes_viagem']].value_counts().reset_index(name='count')
#
df_uf2['ufs_mes'] = df_uf2.uf_origem_linha+'-->'+df_uf2.uf_destino_linha+'-->'+df_uf2.mes_viagem

In [ ]:
fig = plt.figure(figsize=(6, 3.0), dpi = 150, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
temp = df_uf2.iloc[:10]
#temp.plot.bar(x = 'ufs_mes' , y= 'count', edgecolor='black', rot=45, color='red');
sns.barplot(x=temp['count'], y=temp['ufs_mes'], orient='h', color='red')
plt.xlabel('Frequencia')
plt.ylabel('ufs_mes')
plt.title("Top 10\nMaiores Totais de Bilhetes Emitidos Entre Ufs\nOrigem-->Destino--Mes\n");
plt.savefig("Fig9.png",  bbox_inches='tight')

In [ ]:
df_uf3 = df_temp[['uf_origem_linha', 'uf_destino_linha', 'dia_semana_viagem']].value_counts().reset_index(name='count')
#
df_uf3['ufs_dia'] = df_uf3.uf_origem_linha+'-->'+df_uf3.uf_destino_linha+'-->'+df_uf3.dia_semana_viagem

In [ ]:
fig = plt.figure(figsize=(6, 2.0), dpi = 180, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")
temp = df_uf3.iloc[:10]
#temp.plot.bar(x = 'ufs_dia' , y= 'count', edgecolor='black', rot=45, color='red');
sns.barplot(x=temp['count'], y=temp['ufs_dia'], orient='h', color='red')
plt.xlabel('Frequencia')
plt.ylabel('uf_dia')
plt.title("Top 10\nMaiores Totais de Bilhetes Emitidos Entre Ufs\nOrigem-->Destino--Dia\n");
plt.savefig("Fig10.png",  bbox_inches='tight')

In [ ]:
temp = df_temp.copy()

In [ ]:
temp2 = temp.groupby(['uf_origem_linha', 'uf_destino_linha']).value_counts().reset_index(name='counts')


In [ ]:
temp2[['uf_origem_linha', 'uf_destino_linha', 'counts']].sort_values(by='counts', ascending=False)

# Gasto Médio, Min e Max com Bilhete por UF
- tabela: https://www.scaler.com/topics/matplotlib/matplotlib-table/

In [ ]:
df_temp.groupby(['origem_destino_linha'])['valor_total'].agg(['count','sum', 'mean', 'min', 'max'])

In [ ]:
temp_agg = df_temp.copy()
temp_agg['count'] = temp_agg.groupby('origem_destino_linha')['valor_total'].transform('count')
temp_agg['sum'] = np.round(temp_agg.groupby('origem_destino_linha')['valor_total'].transform('sum'),2)
temp_agg['mean'] = np.round(temp_agg.groupby('origem_destino_linha')['valor_total'].transform('mean'),2)
temp_agg['max'] = np.round(temp_agg.groupby('origem_destino_linha')['valor_total'].transform('max'),2)
temp_agg['min'] = np.round(temp_agg.groupby('origem_destino_linha')['valor_total'].transform('min'),2)
temp_agg = temp_agg[['origem_destino_linha','count', 'sum', 'mean', 'max', 'min']]
#
temp_agg.origem_destino_linha = temp_agg.origem_destino_linha.str.replace('-', ' --> ')

In [ ]:
temp_agg.drop_duplicates(inplace=True)
temp_agg = temp_agg.sort_values(by='count', ascending=False)

In [ ]:
fig = plt.figure(figsize=(7, 2.5), dpi = 200, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")

temp_agg = temp_agg[:10]
msg = 'Top 10\nDados Estatisticos de Bilhetes Emitidos\nPeriodo 2019\nOrigem -> Destino -> Linha'

title_text = msg

plt.title(title_text)

fig_background_color = 'grey'
#fig_border = 'skyblue'
headers = temp_agg.columns.to_list()[1:]
#print('headers', headers)
values = temp_agg[['sum', 'mean','max', 'min']].values.tolist()
value_count = temp_agg[ 'count'].values.tolist()
# coluna count deve ser inteira e as demais são decimais ou float
for i in range(len(value_count)):
  values[i].insert(0,value_count[i])

#the headers from the data array
column_headers = headers
row_headers = [x for x in temp_agg.origem_destino_linha.tolist()]

#print('row headers',row_headers)
cell_text = []
for row in values:
    #print('row',row)
    cell_text.append([np.round(x,2) for x in row])

#print('cell_text',cell_text)
rcolors = plt.cm.BuPu(np.full(len(row_headers), 0.4))
ccolors = plt.cm.BuPu(np.full(len(column_headers), 0.4))

#Creating the figure. Setting a small pad on the tight layout

# plt.figure(linewidth=2,
#            edgecolor=fig_border,
#            facecolor = "#EFE9E6",
#            tight_layout={'pad':1})

#Adding a table at the bottom of the axes

the_table = plt.table(cellText=cell_text,
                      rowLabels=row_headers,
                      rowColours=rcolors,
                      rowLoc='center',
                      colColours=ccolors,
                      colLabels=column_headers,
                      loc='center')

# Scaling influences the top and bottom cell padding.
#the_table.scale(2.0, 3.5)

# Hiding axes
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# Hiding axes border
plt.box(on=None)



# Without plt.draw() here, the title will center on the axes and not the figure.
#plt.draw()

# Creating the image. plt.savefig ignores the edge and face colors, so we need to map them.
fig = plt.gcf()
plt.savefig('pyplot-table-demo.png',
            bbox_inches='tight',
            edgecolor=fig.get_edgecolor(),
            facecolor=fig.get_facecolor(),
            dpi=200
            )
plt.savefig("Fig11.png",  bbox_inches='tight')

# Total de Bilhetes Emitidos por UF no periodo de 2019.

In [ ]:
temp = df_uf.copy()
temp['total_uf_orig'] = df_uf.groupby('uf_origem_linha')['count'].transform('sum')


In [ ]:
temp_uf_total = temp[['uf_origem_linha', 'total_uf_orig']].drop_duplicates().sort_values(by='total_uf_orig', ascending=False)

In [ ]:
temp_top10 = temp_uf_total[:10]

In [ ]:
lista_uf = temp_top10.uf_origem_linha.values
lista_uf

In [ ]:
#X= temp_top10['total_uf_orig'].values
#X

In [ ]:
height = temp_top10['total_uf_orig'].reset_index(drop=True)

In [ ]:
rj = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_Rio_de_Janeiro.png')
ba = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_da_Bahia.png')
pb = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_da_Paraíba.png')
al = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Alagoas.png')
#
go = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Goiás.png')
mg = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Minas_Gerais.png')
pe = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Pernambuco.png')
ro = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Rondônia.png')
#
rr = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Roraima.png')
sc = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Santa_Catarina.png')
se = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Sergipe.png')
ac = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Acre.png')
#
ap = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Amapa.png')
am = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Amazonas.png')
ce = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Ceara.png')
df = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Distrito_Federal_Brasil.png')
#
es = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Espírito_Santo.png')
ma = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Maranhao.png')
pa = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Para.png')
pr = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Parana.png')
#
pi = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Piaui.png')
rn = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Rio_Grande_do_Norte.png')
rs = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Rio_Grande_do_Sul.png')
to = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_do_Tocantins.png')
#
sp = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_Sao_Paulo.png')
ms = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_Mato_Grosso_do_Sul.png')
mt = Image.open('/content/drive/MyDrive/Viagens/Bandeiras_Br/Bandeira_de_Mato_Grosso.png')


In [ ]:
dict_icons = {'RJ':rj, 'BA':ba, 'PB':pb, 'AL':al, 'GO':go, 'MG':mg, 'PE':pe, 'RO': ro, 'RR':rr, 'SC': sc, 'SE': se, 'AC':ac, 'AP':ap, 'AM': am, 'CE':ce, 'DF':df,
              'ES': es, 'MA':ma, 'PA': pa, 'PR': pr, 'PI':pi, 'RN': rn, 'RS': rs, 'TO':to, 'SP':sp, 'MS':ms, 'MT':mt}

In [ ]:
X = np.arange(len(height))
X

In [ ]:
temp_top10["uf_origem_linha"]

In [ ]:
fig = plt.figure(figsize=(7, 2.5), dpi = 200, facecolor = "#EFE9E6")
ax = plt.subplot(111, facecolor = "#EFE9E6")

plt.ylabel("total bilhetes")
#plt.xlabel("uf")
plt.title("Top 10\nTotal de Bilhetes Emitidos por UF?\nPeriodo: 2019\n")


# Add spines
ax.spines["top"].set(visible = False)
ax.spines["right"].set(visible = False)

# Add grid and axis labels
ax.grid(True, color = "lightgrey", ls = ":")

# We specify the width of the bar
width = 0.5

# Fouls conceded
ax.bar(
    X,
    height,
    ec = "black",
    lw = .9,
    color = "#005a9b",
    zorder = 3,
    width = width
)

for index, y in enumerate(height):
    ax.annotate(
        xy = (index, y),
        text = f"{y}",
        xytext = (0, 7),
        textcoords = "offset points",
        size = 8,
        color = "black",# "#005a9b",
        ha = "center",
        va = "center",
        weight = "bold"
    )

xticks_ = ax.xaxis.set_ticks(
    ticks = X,
    labels = []
)

ax.tick_params(labelsize = 6)


# --- Axes transformations

DC_to_FC = ax.transData.transform
FC_to_NFC = fig.transFigure.inverted().transform

# Native data to normalized data coordinates
DC_to_NFC = lambda x: FC_to_NFC(DC_to_FC(x))

for index, uf in enumerate(temp_top10["uf_origem_linha"]):
    ax_coords = DC_to_NFC([index - width/2, -0.005])
    logo_ax = fig.add_axes([ax_coords[0], ax_coords[1], 0.07, 0.07], anchor = "W")
    uf_icon = dict_icons.get(uf)
    logo_ax.imshow(uf_icon)
    logo_ax.axis("off")

    logo_ax.annotate(
        xy =(0, 0),
        text = f"{temp_uf_total['uf_origem_linha'].iloc[index]}",
        xytext = (8.5, -23.5), # (x,y) x é a posicao relativa ao eixo vertical e y eixo horizontal onde aparece o icone da UF
        textcoords = "offset points",
        size = 7,
        ha = "center",
        va = "center"
    )

# fig_text(
#     x = 0.12, y = 1.2,
#     s = "Top 10 - Total de Bilhete Emitidos por UF?",
#     weight = "bold",
#     size = 10
# )

# fig_text(
#     x = 0.12, y = 1.1,
#     s = "Periodo: 2019/2020",
#     color = "grey",
#     size = 8
# )
#plt.tight_layout()
plt.subplots_adjust(top=0.70)
plt.savefig("\nTop10_Total_Bilhetes_por_UF.png")
plt.savefig("Fig12.png", bbox_inches='tight')
plt.show()

# INICIAR ANALISE

![](https://drive.google.com/uc?export=view&id=1uRhFT-YkC2B2VHM9FAi-U5Sofkdu7zNU)


# **Análises com base nos gráficos gerados**

In [ ]:
# Fig1
print('\033[1m',"\n\n\tOuarta-feira ficou atrás de sexta-feira e domingo. Feriados ocorrridos na quinta-feira emendando com a sexta-feira \natravés de banco de horas, pode ser uma explicação.\n")
fig = Image.open('/content/Fig1.png')
fig.show()

In [ ]:
# Fig2
print('\033[1m',"\n\n\tA cidade de Campinas no Estado de São Paulo destaca-se entre as mais visitadas, as demais são todas capitais e o distrito federal.\n\tNa região de Campinas se localizam a Unicamp, centros de pesquisas como o CPQD e grandes empresas tais como a IBM e EMS.\n")
fig = Image.open('/content/Fig2.png')
fig.show()

In [ ]:
# Fig3
#print(df5['ano_viagem'].value_counts(normalize=True))
"""
2019    0.720453
2020    0.279547
Name: ano_viagem, dtype: float64
"""
print('\033[1m',"\n\n\tNa base de dados o ano de 2020 representa 28%, sendo um ano marcado pelas restrições de deslocamento devido a pandemia do Covid.\n\tOs meses de Dezembro, Novembro e Março mostram os valores relativos a 2019, pois não haviam restrições ainda.\n\n")
fig = Image.open('/content/Fig3.png')
fig.show()

In [ ]:
# Fig4
#print(df5['tipo_gratitude'].value_counts(normalize=True))
"""
Tarifa Normal - sem desconto                                                     0.708986
Tarifa Promocional - Parágrafo 3º, art. 27 do Decreto nº 2.521/98                0.217253
Bilhete de Viagem do Idoso 50% - Inciso II, art. 40 da Lei nº 10.741/03          0.033761
Bilhete de Viagem do Idoso 100% - Inciso I, art. 40 da Lei nº 10.741/03          0.013223
Gratuidade Jovem de Baixa Renda 100% - Inciso I, art. 32 da Lei nº 12.852/13     0.009013
Autorização de Viagem - Passe Livre - Art. 1º da Lei nº 8.899/94                 0.007683
Gratuidade Jovem de Baixa Renda 50% - Inciso II, art. 32 da Lei nº 12.852/13     0.006179
Gratuidade de Criança - Inciso XVII, art. 29 do Decreto nº 2.521/98              0.003333
Passe Livre Auditores e Agentes do Trabalho - Art. 34 do Decreto nº 4.552/02     0.000568
Autorização de Viagem - Passe Livre Acompanhante - Art. 1º da Lei nº 8.899/94    0.000001
Name: tipo_gratitude, dtype: float64
"""
print('\033[1m',"\n\n\tA tarifa normal, sem desconto representa 70% e o restante corresponde ao uso de algum beneficio social ou promoção.\n")
fig = Image.open('/content/Fig4.png')
fig.show()

In [ ]:
# Fig5
print('\033[1m',"\n\n\tNatal e ano novo são os maiores responsáveis por bilhetes emitidos, provavelmente comprados com antecedência devido ao grande fluxo no periodo.\n\n")
fig = Image.open('/content/Fig5.png')
fig.show()

In [ ]:
# Fig6
print('\033[1m',"\n\n\tO Estado de São Paulo esta presente como destino, em 4 dos 10 Top maiores volumes de bilhetes emitidos.\n\tO que indica ser um ponto de atração para diversas pessoas.\n\tQue visitam por motivo de trabalho, familiar ou a passeio.\n\n")
fig = Image.open('/content/Fig6.png')
fig.show()

In [ ]:
#df_uf.loc[df_uf['count']>= filter].sort_values(by='count', ascending=False)[['uf_origem_linha', 'uf_destino_linha', 'count']].head()

In [ ]:
# Fig7
# df_uf.loc[df_uf['count']>= filter].sort_values(by='count', ascending=False)[['uf_origem_linha', 'uf_destino_linha', 'count']]
"""
	uf_origem_linha	uf_destino_linha	count
0	PR	SP	154065
1	MG	RJ	134074
2	SP	MG	133341
3	SP	RJ	127959
-
-
-
35	AL	PE	10547
36	BA	RJ	10502
"""
print('\033[1m',"\n\n\tOs maiores fluxos de pessoas viajando saem do Paraná, São Paulo, Minas Gerais, Rio de Janeiro, Espirito Santo e Bahia.\n\n")
fig = Image.open('/content/Fig7.png')
fig.show()

In [ ]:
# Fig8
print('\033[1m',"\n\n\tNo mês de Março de 2019 ocorreu o carnaval e provavelmente muitas pessoas viajaram.\n\tEm 2020 haviam restrições devido ao Covid.\n\n")
fig = Image.open('/content/Fig8.png')
fig.show()

In [ ]:
# Fig9
print('\033[1m',"\n\n\tOs Estados do Paraná e São Paulo apresentam grande fluxo de viajantes e a maioria deles prefere viajar na sexta-feira ou domingo.\n\n")
fig = Image.open('/content/Fig9.png')
fig.show()

In [ ]:
# Fig10
print('\033[1m',"\n\n\tA tabela mostra as linhas com maiores bilhetes emitidos.\n\tAs cidades: São Paulo, Curitiba e Belo Horizonte,  surgem como grandes fontes de viagens entre capitais.\n\n")
fig = Image.open('/content/Fig10.png')
fig.show()

In [ ]:
# Fig11
print('\033[1m',"\n\n\tOs Estados de São Paulo, Paraná e Minas Gerais respondem por um enorme fluxo de viagens.\n\n")
fig = Image.open('/content/Fig11.png')
fig.show()